# AnimateDiff - Introduction a la Generation Text-to-Video

**Module :** 01-Video-Foundation  
**Niveau :** Intermediaire  
**Technologies :** AnimateDiff, diffusers, Stable Diffusion v1.5  
**Duree estimee :** 45 minutes  
**VRAM :** ~12 GB  

## Objectifs d'Apprentissage

- [ ] Comprendre l'architecture AnimateDiff (motion module + base model)
- [ ] Charger un pipeline AnimateDiff avec la bibliotheque diffusers
- [ ] Generer des videos a partir de descriptions textuelles
- [ ] Explorer les parametres cles (num_frames, guidance_scale, num_inference_steps)
- [ ] Sauvegarder les resultats en GIF et MP4
- [ ] Comparer differents prompts et parametres

## Prerequis

- GPU avec 12+ GB VRAM (RTX 3060 12GB minimum)
- Notebooks 01-1 a 01-4 completes
- Packages : `diffusers>=0.30`, `transformers`, `torch`, `accelerate`, `imageio`, `imageio-ffmpeg`

## Principe d'AnimateDiff

AnimateDiff ajoute un **motion module** (attention temporelle) a un modele Stable Diffusion
existant pour generer des sequences de frames coherentes temporellement.

| Composant | Role |
|-----------|------|
| **Base model** | Stable Diffusion v1.5 (generation d'images) |
| **Motion adapter** | Module d'attention temporelle (coherence entre frames) |
| **Scheduler** | Controle du processus de debruitage (DDIM, Euler, etc.) |

In [1]:
# Parametres Papermill - JAMAIS modifier ce commentaire

# Configuration notebook
notebook_mode = "interactive"        # "interactive" ou "batch"
skip_widgets = False               # True pour mode batch MCP
debug_level = "INFO"

# Parametres modele
base_model = "stable-diffusion-v1-5/stable-diffusion-v1-5"  # Modele de base SD
motion_adapter = "guoyww/animatediff-motion-adapter-v1-5-3"  # Motion module

# Parametres generation
num_frames = 16                    # Nombre de frames a generer
guidance_scale = 7.5               # CFG scale (adherence au prompt)
num_inference_steps = 25           # Nombre d'etapes de debruitage
height = 512                       # Hauteur video
width = 512                        # Largeur video
fps_output = 8                     # FPS de la video de sortie

# Configuration
run_generation = True              # Executer la generation
save_as_gif = True                 # Sauvegarder en GIF
save_as_mp4 = True                 # Sauvegarder en MP4
save_results = True

In [2]:
# Parameters
notebook_mode = "batch"
skip_widgets = True


In [3]:
# Setup environnement et imports
import os
import sys
import json
import time
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import logging

warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Import helpers GenAI
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.genai_helpers import setup_genai_logging
        print("Helpers GenAI importes")
    except ImportError:
        print("Helpers GenAI non disponibles - mode autonome")

OUTPUT_DIR = GENAI_ROOT / 'outputs' / 'animatediff'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('animatediff')

print(f"AnimateDiff - Generation Text-to-Video")
print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}")
print(f"Frames : {num_frames}, Steps : {num_inference_steps}, CFG : {guidance_scale}")

AnimateDiff - Generation Text-to-Video
Date : 2026-02-19 10:29:13
Mode : batch
Frames : 16, Steps : 25, CFG : 7.5


In [4]:
# Chargement .env et verification GPU
from dotenv import load_dotenv

current_path = Path.cwd()
found_env = False
for _ in range(4):
    env_path = current_path / '.env'
    if env_path.exists():
        load_dotenv(env_path)
        print(f"Fichier .env charge depuis : {env_path}")
        found_env = True
        break
    current_path = current_path.parent

if not found_env:
    print("Aucun fichier .env trouve")

# Verification GPU
print("\n--- VERIFICATION GPU ---")
print("=" * 40)

import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram_total = torch.cuda.get_device_properties(0).total_mem / 1024**3
    vram_free = (torch.cuda.get_device_properties(0).total_mem - torch.cuda.memory_allocated(0)) / 1024**3
    
    print(f"GPU : {gpu_name}")
    print(f"VRAM totale : {vram_total:.1f} GB")
    print(f"VRAM libre : {vram_free:.1f} GB")
    print(f"CUDA : {torch.version.cuda}")
    
    if vram_total < 12:
        print(f"\nAttention : VRAM ({vram_total:.0f} GB) < 12 GB recommandes")
        print("Reduction de la resolution et du nombre de frames")
        if vram_total < 8:
            height = 256
            width = 256
            num_frames = 8
            print(f"  Resolution reduite a {width}x{height}, {num_frames} frames")
else:
    print("CUDA non disponible.")
    print("AnimateDiff necessite un GPU. Le notebook montrera le code sans executer.")
    run_generation = False

# Verification des dependances
print("\n--- VERIFICATION DEPENDANCES ---")
print("=" * 40)

# Verification huggingface-hub version AVANT d'importer diffusers
animatediff_available = False
try:
    import huggingface_hub
    hf_version = huggingface_hub.__version__
    print(f"huggingface-hub : v{hf_version}")
    # Verifier si la version est compatible
    from packaging import version
    if version.parse(hf_version) >= version.parse("1.0"):
        print(f"  Warning: huggingface-hub v{hf_version} n'est pas compatible avec diffusers")
        print(f"  AnimateDiff ne sera pas disponible (conflit de version)")
        animatediff_available = False
    else:
        animatediff_available = True
except ImportError:
    print("huggingface-hub NON INSTALLE")
    animatediff_available = False
except Exception as e:
    print(f"Erreur verification huggingface-hub: {e}")
    animatediff_available = False

# Verification diffusers
try:
    import diffusers
    print(f"diffusers : v{diffusers.__version__}")
except ImportError:
    print("diffusers NON INSTALLE")
    animatediff_available = False

# Import AnimateDiff seulement si les dependances sont OK
if animatediff_available:
    try:
        from diffusers import AnimateDiffPipeline, MotionAdapter, DDIMScheduler
        from diffusers.utils import export_to_gif, export_to_video
        print("AnimateDiff pipeline : disponible")
    except ImportError as e:
        error_str = str(e)
        if "huggingface-hub" in error_str:
            print(f"AnimateDiff pipeline : NON DISPONIBLE (conflit de version huggingface-hub)")
            print(f"  Solution: pip install 'huggingface-hub<1.0'")
        else:
            print(f"AnimateDiff pipeline : NON DISPONIBLE - {error_str[:100]}")
        animatediff_available = False
        run_generation = False
else:
    if not animatediff_available:
        print("AnimateDiff pipeline : NON DISPONIBLE (dependances)")
    run_generation = False

print(f"\nDevice : {device}")
print(f"Generation activee : {run_generation}")

Aucun fichier .env trouve

--- VERIFICATION GPU ---


CUDA non disponible.
AnimateDiff necessite un GPU. Le notebook montrera le code sans executer.

--- VERIFICATION DEPENDANCES ---
huggingface-hub : v1.4.1
  AnimateDiff ne sera pas disponible (conflit de version)


diffusers : v0.36.0
AnimateDiff pipeline : NON DISPONIBLE (dependances)

Device : cpu
Generation activee : False


## Section 1 : Chargement du pipeline AnimateDiff

Le pipeline AnimateDiff combine :
1. Un **motion adapter** (module d'attention temporelle entraine separement)
2. Un **modele Stable Diffusion** de base (genere les frames individuelles)
3. Un **scheduler** (controle le processus de debruitage)

Le motion adapter s'insere dans les couches d'attention du U-Net de Stable Diffusion
pour ajouter une dimension temporelle au traitement.

In [5]:
# Chargement du pipeline AnimateDiff
pipe = None

if run_generation:
    print("\n--- CHARGEMENT DU PIPELINE ---")
    print("=" * 40)
    
    try:
        # 1. Charger le motion adapter
        print(f"Chargement motion adapter : {motion_adapter}")
        adapter = MotionAdapter.from_pretrained(
            motion_adapter,
            torch_dtype=torch.float16
        )
        print(f"  Motion adapter charge")
        
        # 2. Charger le pipeline complet
        print(f"Chargement base model : {base_model}")
        start_load = time.time()
        
        pipe = AnimateDiffPipeline.from_pretrained(
            base_model,
            motion_adapter=adapter,
            torch_dtype=torch.float16
        )
        
        # 3. Configurer le scheduler
        pipe.scheduler = DDIMScheduler.from_pretrained(
            base_model,
            subfolder="scheduler",
            clip_sample=False,
            timestep_spacing="linspace",
            beta_schedule="linear",
            steps_offset=1
        )
        
        # 4. Transferer sur GPU
        pipe = pipe.to(device)
        
        # 5. Optimisations memoire
        pipe.enable_vae_slicing()  # Reduit la VRAM pour le decodage VAE
        try:
            pipe.enable_model_cpu_offload()  # Offload intelligent CPU<->GPU
        except Exception:
            pass  # Pas critique si indisponible
        
        load_time = time.time() - start_load
        
        if device == "cuda":
            vram_used = torch.cuda.memory_allocated(0) / 1024**3
            print(f"  VRAM utilisee : {vram_used:.1f} GB")
        
        print(f"Pipeline charge en {load_time:.1f}s")
        print(f"  Scheduler : DDIMScheduler")
        print(f"  VAE slicing : active")
        print(f"  Resolution : {width}x{height}")
        
    except Exception as e:
        print(f"Erreur chargement pipeline : {type(e).__name__}: {str(e)[:200]}")
        print("Le notebook continuera sans generation.")
        run_generation = False
        pipe = None
else:
    print("Chargement pipeline desactive")

Chargement pipeline desactive


## Section 2 : Generation text-to-video

Nous allons generer une premiere video a partir d'un prompt textuel. Les parametres
principaux sont :

| Parametre | Impact | Valeur typique |
|-----------|--------|---------------|
| `num_frames` | Duree de la video | 8-24 |
| `guidance_scale` | Adherence au prompt (trop haut = artefacts) | 5.0-10.0 |
| `num_inference_steps` | Qualite (plus = mieux mais plus lent) | 20-50 |
| `height/width` | Resolution (carree recommandee) | 256-512 |

In [6]:
# Generation text-to-video
print("\n--- GENERATION TEXT-TO-VIDEO ---")
print("=" * 40)

def generate_video(prompt: str, negative_prompt: str = "",
                   seed: int = 42) -> Dict[str, Any]:
    """
    Genere une video a partir d'un prompt textuel.
    
    Args:
        prompt: Description textuelle de la video
        negative_prompt: Elements a eviter
        seed: Graine aleatoire pour reproductibilite
    
    Returns:
        Dict avec frames, temps de generation et metadonnees
    """
    if pipe is None:
        return {"success": False, "error": "Pipeline non charge"}
    
    try:
        generator = torch.Generator(device=device).manual_seed(seed)
        
        if device == "cuda":
            torch.cuda.reset_peak_memory_stats()
            vram_before = torch.cuda.memory_allocated(0) / 1024**3
        
        start_time = time.time()
        
        output = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt or "low quality, blurry, distorted",
            num_frames=num_frames,
            guidance_scale=guidance_scale,
            num_inference_steps=num_inference_steps,
            height=height,
            width=width,
            generator=generator
        )
        
        gen_time = time.time() - start_time
        frames = output.frames[0]  # Liste d'images PIL
        
        result = {
            "success": True,
            "frames": frames,
            "generation_time": gen_time,
            "time_per_frame": gen_time / num_frames,
            "prompt": prompt,
            "seed": seed,
            "params": {
                "num_frames": num_frames,
                "guidance_scale": guidance_scale,
                "num_inference_steps": num_inference_steps,
                "height": height,
                "width": width
            }
        }
        
        if device == "cuda":
            result["vram_peak"] = torch.cuda.max_memory_allocated(0) / 1024**3
        
        return result
        
    except Exception as e:
        return {"success": False, "error": f"{type(e).__name__}: {str(e)[:200]}"}


# Premier test : generation simple
prompt_1 = "a serene lake at sunset with mountains in the background, golden light reflecting on water, cinematic"

if run_generation:
    print(f"Prompt : {prompt_1}")
    print(f"Parametres : {num_frames} frames, {num_inference_steps} steps, CFG={guidance_scale}")
    print(f"Resolution : {width}x{height}")
    print(f"\nGeneration en cours...")
    
    result_1 = generate_video(prompt_1, seed=42)
    
    if result_1['success']:
        frames = result_1['frames']
        print(f"\nGeneration reussie")
        print(f"  Temps total : {result_1['generation_time']:.1f}s")
        print(f"  Temps/frame : {result_1['time_per_frame']:.1f}s")
        print(f"  Frames : {len(frames)}")
        if 'vram_peak' in result_1:
            print(f"  VRAM pic : {result_1['vram_peak']:.1f} GB")
        
        # Affichage en grille
        n_display = min(8, len(frames))
        indices = np.linspace(0, len(frames) - 1, n_display, dtype=int)
        fig, axes = plt.subplots(2, 4, figsize=(16, 8))
        axes_flat = axes.flatten()
        for i, idx in enumerate(indices):
            if i < len(axes_flat):
                axes_flat[i].imshow(frames[idx])
                axes_flat[i].set_title(f"Frame {idx + 1}/{len(frames)}", fontsize=9)
                axes_flat[i].axis('off')
        for i in range(len(indices), len(axes_flat)):
            axes_flat[i].axis('off')
        plt.suptitle(f"AnimateDiff : {prompt_1[:60]}...", fontsize=11, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        # Sauvegarde GIF
        if save_as_gif:
            gif_path = OUTPUT_DIR / "animatediff_demo.gif"
            export_to_gif(frames, str(gif_path))
            gif_size_kb = gif_path.stat().st_size / 1024
            print(f"  GIF sauvegarde : {gif_path.name} ({gif_size_kb:.1f} KB)")
        
        # Sauvegarde MP4
        if save_as_mp4:
            mp4_path = OUTPUT_DIR / "animatediff_demo.mp4"
            export_to_video(frames, str(mp4_path), fps=fps_output)
            mp4_size_kb = mp4_path.stat().st_size / 1024
            print(f"  MP4 sauvegarde : {mp4_path.name} ({mp4_size_kb:.1f} KB)")
    else:
        print(f"Erreur : {result_1['error']}")
else:
    # Mode pedagogique : afficher ce qui serait genere
    print("**MODE PEDAGOGIQUE**")
    print(f"\nSur un environnement GPU (RTX 3090, 24GB VRAM), ce code genererait :")
    print(f"")
    print(f"| Parametre | Valeur |")
    print(f"|-----------|--------|")
    print(f"| Prompt | {prompt_1[:50]}... |")
    print(f"| Frames | {num_frames} |")
    print(f"| Resolution | {width}x{height} |")
    print(f"| Steps | {num_inference_steps} |")
    print(f"| CFG Scale | {guidance_scale} |")
    print(f"| Temps estime | ~45s |")
    print(f"| VRAM | ~10 GB |")
    print(f"")
    print(f"**Resultat attendu :**")
    print(f"Une video de 2 secondes (16 frames @ 8 fps) montrant un lac paisible")
    print(f"au coucher du soleil avec des montagnes en arriere-plan et des")
    print(f"reflets dorees sur l'eau. Le mouvement sera subtil : ondulations de")
    print(f"l'eau, nuages se deplacant lentement, lumiere changeante.")
    print(f"")
    print(f"**Code pour reproduire :**")
    print(f"```python")
    print(f"result = generate_video('{prompt_1[:40]}...', seed=42)")
    print(f"```")


--- GENERATION TEXT-TO-VIDEO ---
**MODE PEDAGOGIQUE**

Sur un environnement GPU (RTX 3090, 24GB VRAM), ce code genererait :

| Parametre | Valeur |
|-----------|--------|
| Prompt | a serene lake at sunset with mountains in the back... |
| Frames | 16 |
| Resolution | 512x512 |
| Steps | 25 |
| CFG Scale | 7.5 |
| Temps estime | ~45s |
| VRAM | ~10 GB |

**Resultat attendu :**
Une video de 2 secondes (16 frames @ 8 fps) montrant un lac paisible
au coucher du soleil avec des montagnes en arriere-plan et des
reflets dorees sur l'eau. Le mouvement sera subtil : ondulations de
l'eau, nuages se deplacant lentement, lumiere changeante.

**Code pour reproduire :**
```python
result = generate_video('a serene lake at sunset with mountains i...', seed=42)
```


## Section 3 : Exploration des parametres

Nous allons tester differentes combinaisons de parametres pour comprendre leur impact
sur la qualite et le temps de generation.

In [7]:
# Comparaison de prompts
if run_generation and pipe is not None:
    print("\n--- COMPARAISON DE PROMPTS ---")
    print("=" * 40)
    
    prompts = [
        {
            "text": "a cat walking gracefully on a windowsill, soft morning light, cozy atmosphere",
            "label": "Chat"
        },
        {
            "text": "ocean waves crashing on a rocky shore, dramatic clouds, powerful nature",
            "label": "Ocean"
        },
        {
            "text": "a rocket launching into space with fire and smoke, cinematic epic shot",
            "label": "Fusee"
        }
    ]
    
    comparison_results = []
    
    for p_idx, prompt_info in enumerate(prompts):
        print(f"\nGeneration {p_idx + 1}/{len(prompts)} : {prompt_info['label']}")
        print(f"  Prompt : {prompt_info['text'][:70]}...")
        
        result = generate_video(prompt_info['text'], seed=42 + p_idx)
        
        if result['success']:
            print(f"  Temps : {result['generation_time']:.1f}s")
            comparison_results.append({
                "label": prompt_info['label'],
                "prompt": prompt_info['text'],
                "frames": result['frames'],
                "time": result['generation_time']
            })
            
            # Sauvegarder chaque video
            if save_as_gif:
                gif_path = OUTPUT_DIR / f"comparison_{prompt_info['label'].lower()}.gif"
                export_to_gif(result['frames'], str(gif_path))
        else:
            print(f"  Erreur : {result['error']}")
    
    # Affichage comparatif
    if comparison_results:
        n_videos = len(comparison_results)
        n_preview = 4
        fig, axes = plt.subplots(n_videos, n_preview, figsize=(3.5 * n_preview, 3 * n_videos))
        if n_videos == 1:
            axes = [axes]
        
        for v_idx, cr in enumerate(comparison_results):
            frame_indices = np.linspace(0, len(cr['frames']) - 1, n_preview, dtype=int)
            for f_idx, fi in enumerate(frame_indices):
                axes[v_idx][f_idx].imshow(cr['frames'][fi])
                axes[v_idx][f_idx].axis('off')
                if f_idx == 0:
                    axes[v_idx][f_idx].set_ylabel(cr['label'], fontsize=11, fontweight='bold')
        
        plt.suptitle("Comparaison de prompts - AnimateDiff", fontsize=13, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        # Tableau recapitulatif
        print(f"\nRecapitulatif :")
        print(f"{'Label':<15} {'Temps (s)':<12} {'Prompt':<60}")
        print("-" * 85)
        for cr in comparison_results:
            prompt_short = cr['prompt'][:55] + '...' if len(cr['prompt']) > 55 else cr['prompt']
            print(f"  {cr['label']:<15} {cr['time']:<12.1f} {prompt_short:<60}")
else:
    print("Comparaison de prompts : generation desactivee")
    print("\nExemples de prompts efficaces pour AnimateDiff :")
    print("  - Mouvements naturels : eau, vent, nuages")
    print("  - Animaux en mouvement : chat marchant, oiseau volant")
    print("  - Paysages dynamiques : coucher de soleil, aurores boreales")
    print("  - Actions simples : fusee decollant, fleur eclosant")

Comparaison de prompts : generation desactivee

Exemples de prompts efficaces pour AnimateDiff :
  - Mouvements naturels : eau, vent, nuages
  - Animaux en mouvement : chat marchant, oiseau volant
  - Paysages dynamiques : coucher de soleil, aurores boreales
  - Actions simples : fusee decollant, fleur eclosant


In [8]:
# Mode interactif
if notebook_mode == "interactive" and not skip_widgets:
    print("\n--- MODE INTERACTIF ---")
    print("=" * 40)
    print("Entrez votre propre prompt pour generer une video.")
    print("(Laissez vide pour passer a la suite)")
    
    try:
        user_prompt = input("\nVotre prompt : ").strip()
        
        if user_prompt and run_generation and pipe is not None:
            print(f"\nGeneration en cours...")
            result_user = generate_video(user_prompt, seed=123)
            
            if result_user['success']:
                print(f"Generation reussie en {result_user['generation_time']:.1f}s")
                
                # Affichage
                n_display = min(8, len(result_user['frames']))
                fig, axes = plt.subplots(1, n_display, figsize=(2.5 * n_display, 3))
                if n_display == 1:
                    axes = [axes]
                indices = np.linspace(0, len(result_user['frames']) - 1, n_display, dtype=int)
                for ax, idx in zip(axes, indices):
                    ax.imshow(result_user['frames'][idx])
                    ax.set_title(f"Frame {idx+1}", fontsize=8)
                    ax.axis('off')
                plt.suptitle(f"Votre video : {user_prompt[:50]}...", fontweight='bold')
                plt.tight_layout()
                plt.show()
                
                if save_as_gif:
                    user_gif = OUTPUT_DIR / "user_generation.gif"
                    export_to_gif(result_user['frames'], str(user_gif))
                    print(f"GIF sauvegarde : {user_gif.name}")
            else:
                print(f"Erreur : {result_user['error']}")
        elif user_prompt:
            print("Generation non disponible (pipeline non charge)")
        else:
            print("Mode interactif ignore")
    
    except (KeyboardInterrupt, EOFError) as e:
        print(f"\nMode interactif interrompu ({type(e).__name__})")
    except Exception as e:
        error_type = type(e).__name__
        if "StdinNotImplemented" in error_type or "input" in str(e).lower():
            print("\nMode interactif non disponible (execution automatisee)")
        else:
            print(f"\nErreur inattendue : {error_type} - {str(e)[:100]}")
            print("Passage a la suite du notebook")
else:
    print("\nMode batch - Interface interactive desactivee")


Mode batch - Interface interactive desactivee


## Bonnes pratiques et optimisation

### Conseils pour les prompts AnimateDiff

| Bon prompt | Mauvais prompt | Raison |
|-----------|---------------|--------|
| "a cat walking slowly" | "a cat" | Preciser le mouvement |
| "ocean waves, cinematic" | "nice ocean" | Termes de qualite |
| "single subject, simple motion" | "crowd scene, complex action" | Mouvement simple = meilleur |

### Parametres optimaux

| Parametre | Rapide | Equilibre | Haute qualite |
|-----------|--------|-----------|---------------|
| `num_frames` | 8 | 16 | 24 |
| `num_inference_steps` | 15 | 25 | 50 |
| `guidance_scale` | 5.0 | 7.5 | 7.5 |
| `height/width` | 256 | 512 | 512 |
| Temps estime (RTX 3090) | ~15s | ~45s | ~120s |
| VRAM estimee | ~6 GB | ~10 GB | ~14 GB |

### Limitations d'AnimateDiff

- Resolution maximale limitee (512x512 pour SD 1.5)
- Mouvements complexes souvent mal geres
- Coherence temporelle imparfaite sur les longues sequences
- Base model SD 1.5 (qualite inferieure a SDXL ou Flux)

### Alternatives (voir Module 02)

| Modele | Avantage | VRAM |
|--------|----------|------|
| **HunyuanVideo** | Haute qualite, longues sequences | ~18 GB |
| **LTX-Video** | Rapide, leger | ~8 GB |
| **Wan 2.1/2.2** | Prompts multilingues | ~10 GB |
| **SVD** | Image-to-video | ~10 GB |

In [9]:
# Statistiques de session et prochaines etapes
print("\n--- STATISTIQUES DE SESSION ---")
print("=" * 40)

print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}")
print(f"Base model : {base_model}")
print(f"Motion adapter : {motion_adapter}")
print(f"Device : {device}")
print(f"Parametres : {num_frames} frames, {num_inference_steps} steps, CFG={guidance_scale}")
print(f"Resolution : {width}x{height}")

if device == "cuda" and torch.cuda.is_available():
    vram_peak = torch.cuda.max_memory_allocated(0) / 1024**3
    print(f"VRAM pic session : {vram_peak:.1f} GB")

if save_results and OUTPUT_DIR.exists():
    generated_files = list(OUTPUT_DIR.glob('*'))
    print(f"\nFichiers generes ({len(generated_files)}) :")
    for f in sorted(generated_files):
        size_kb = f.stat().st_size / 1024
        print(f"  {f.name} ({size_kb:.1f} KB)")

# Liberation VRAM
if pipe is not None:
    del pipe
    if device == "cuda":
        torch.cuda.empty_cache()
        print(f"\nVRAM liberee")

print(f"\n--- PROCHAINES ETAPES ---")
print(f"1. Module 02-1 : HunyuanVideo (generation video haute qualite)")
print(f"2. Module 02-2 : LTX-Video (generation rapide et legere)")
print(f"3. Module 02-3 : Wan 2.1/2.2 (prompts multilingues)")
print(f"4. Module 02-4 : SVD - Stable Video Diffusion (image-to-video)")
print(f"5. Combiner 01-4 (upscaling) avec la generation pour des videos HD")

print(f"\nNotebook 01-5 AnimateDiff Introduction termine - {datetime.now().strftime('%H:%M:%S')}")


--- STATISTIQUES DE SESSION ---
Date : 2026-02-19 10:29:25
Mode : batch
Base model : stable-diffusion-v1-5/stable-diffusion-v1-5
Motion adapter : guoyww/animatediff-motion-adapter-v1-5-3
Device : cpu
Parametres : 16 frames, 25 steps, CFG=7.5
Resolution : 512x512

Fichiers generes (0) :

--- PROCHAINES ETAPES ---
1. Module 02-1 : HunyuanVideo (generation video haute qualite)
2. Module 02-2 : LTX-Video (generation rapide et legere)
3. Module 02-3 : Wan 2.1/2.2 (prompts multilingues)
4. Module 02-4 : SVD - Stable Video Diffusion (image-to-video)
5. Combiner 01-4 (upscaling) avec la generation pour des videos HD

Notebook 01-5 AnimateDiff Introduction termine - 10:29:25
